In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [9]:
df = pd.DataFrame(row)
data = df[0].str.split('\n', expand=True)
data = data.rename(columns=data.iloc[0])
data = data.drop(data.index[0])

data.drop(data[data['Borough']=='Not assigned'].index,inplace=True)

data = data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()


for index, row in data.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

### Fetching geographical coordinates of each postal code

In [10]:
url = "http://cocl.us/Geospatial_data"
new_df=pd.read_csv(url)
new_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Renaming Columns

In [11]:
new_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
new_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging Dataframes

In [12]:
df_tornoto = pd.merge(data, new_df, on='Postcode')
df_tornoto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
